In [2]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import math

print('Libraries imported.')

Libraries imported.


In [4]:
# The code was removed by Watson Studio for sharing.

,id,distribuidor,comuna,latitud,longitud,habitantes,viviendas,vehículos,calidad
0,pe01001,Petrobras,Santiago Centro,-33.453465,-70.644193,404495,193628,59900,59.27
1,pe01002,Petrobras,Santiago Centro,-33.430747,-70.665559,404495,193628,59900,59.27
2,pe01003,Petrobras,Santiago Centro,-33.445162,-70.640700,404495,193628,59900,59.27
3,pe01004,Petrobras,Santiago Centro,-33.442951,-70.636680,404495,193628,59900,59.27
4,pe01006,Petrobras,Santiago Centro,-33.459715,-70.652395,404495,193628,59900,59.27


In [5]:
address = 'Santiago, Chile'

geolocator = Nominatim(user_agent="eds_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Santiago, Chile are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Santiago, Chile are -33.4377968, -70.6504451.


In [6]:
# create map of Toronto using latitude and longitude values
map_chile = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, distribuidor, id_estacion in zip(df['latitud'], 
                                           df['longitud'], 
                                           df['distribuidor'], 
                                           df['id']):
    label = '{}, {}'.format(distribuidor, id_estacion)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chile)  
    
map_chile

In [7]:
CLIENT_ID = 'HHO52RQYVPEQNRYR3PVO5B1ORIEBJWE2GDPJ4JUYXTPIS1A5' # your Foursquare ID
CLIENT_SECRET = 'U4K5GA1E0Z33LW1RPQ10Y2JFDGLUMDQ0FF2TXGOIBPDH1KVP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HHO52RQYVPEQNRYR3PVO5B1ORIEBJWE2GDPJ4JUYXTPIS1A5
CLIENT_SECRET:U4K5GA1E0Z33LW1RPQ10Y2JFDGLUMDQ0FF2TXGOIBPDH1KVP


In [8]:


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
RADIUS = 500 # define radius

venues_list=[]

for latitud, longitud, distribuidor, id_estacion in zip(df['latitud'], 
                                           df['longitud'], 
                                           df['distribuidor'], 
                                           df['id']):

    # create URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitud, 
        longitud, 
        RADIUS, 
        LIMIT)
    
    #print(url)
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        id_estacion, 
        latitud, 
        longitud, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['ID Station', 
    'Neighborhood Latitude', 
    'Neighborhood Longitude', 
    'Venue', 
    'Venue Latitude', 
    'Venue Longitude', 
    'Venue Category']

nearby_venues.head()

,ID Station,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,pe01001,-33.453465,-70.644193,Pollos asados m&m,-33.451456,-70.642742,Fried Chicken Joint
1,pe01001,-33.453465,-70.644193,Plaza Sacramentinos,-33.451312,-70.648375,Plaza
2,pe01001,-33.453465,-70.644193,Alto Japón,-33.451181,-70.646762,Sushi Restaurant
3,pe01001,-33.453465,-70.644193,Crossfighter,-33.452239,-70.646607,Gym / Fitness Center
4,pe01001,-33.453465,-70.644193,Milano‘s Pizza,-33.450126,-70.642222,Pizza Place


In [40]:
nearby_venues.groupby('ID Station').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ID Station,,,,,,
co1360501,5,5,5,5,5,5
pe01001,29,29,29,29,29,29
pe01002,13,13,13,13,13,13
pe01003,44,44,44,44,44,44
pe01004,81,81,81,81,81,81
pe01006,15,15,15,15,15,15
pe01008,28,28,28,28,28,28
pe1310201,5,5,5,5,5,5
pe1310401,37,37,37,37,37,37


In [75]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 216 uniques categories.


In [76]:
# one hot encoding
distribuidor_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
distribuidor_onehot['ID Station'] = nearby_venues['ID Station'] 

aux = distribuidor_onehot['ID Station']
distribuidor_onehot.drop(labels=['ID Station'], axis=1, inplace=True)
distribuidor_onehot.insert(0, 'ID Station', aux)

distribuidor_onehot.head()

,ID Station,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,pe01001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,pe01001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,pe01001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [77]:
distribuidor_onehot.shape

(1562, 217)

In [78]:
distribuidor_grouped = distribuidor_onehot.groupby('ID Station').mean().reset_index()
distribuidor_grouped

,ID Station,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,co1360501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [79]:
distribuidor_grouped.shape

(92, 217)

In [80]:
num_top_venues = 5

for hood in distribuidor_grouped['ID Station']:
    print("----"+hood+"----")
    temp = distribuidor_grouped[distribuidor_grouped['ID Station'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----co1360501----
                    venue  freq
0  Furniture / Home Store   0.2
1          Breakfast Spot   0.2
2                   Plaza   0.2
3             Pizza Place   0.2
4      Chinese Restaurant   0.2


----pe01001----
              venue  freq
0     Grocery Store  0.10
1  Sushi Restaurant  0.10
2        Restaurant  0.07
3       Pizza Place  0.07
4        Print Shop  0.03


----pe01002----
              venue  freq
0              Park  0.15
1      Soccer Field  0.15
2  Sushi Restaurant  0.15
3    Farmers Market  0.08
4        Skate Park  0.08


----pe01003----
              venue  freq
0        Restaurant  0.09
1              Café  0.07
2  Sushi Restaurant  0.07
3             Plaza  0.07
4       Coffee Shop  0.07


----pe01004----
              venue  freq
0  Sushi Restaurant  0.06
1              Café  0.05
2       Coffee Shop  0.05
3        Restaurant  0.05
4            Bakery  0.05


----pe01006----
                           venue  freq
0                     Restaurant  0.1

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ID Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['ID Station'] = distribuidor_grouped['ID Station']

for ind in np.arange(distribuidor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(distribuidor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,ID Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,co1360501,Plaza,Breakfast Spot,Pizza Place,Furniture / Home Store,Chinese Restaurant,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market
1,pe01001,Grocery Store,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Rock Club,Cupcake Shop,Latin American Restaurant,Fried Chicken Joint,Flea Market
2,pe01002,Sushi Restaurant,Soccer Field,Park,Farmers Market,Bakery,River,Chinese Restaurant,BBQ Joint,Skate Park,Antique Shop
3,pe01003,Restaurant,Café,Sushi Restaurant,Pizza Place,Plaza,Coffee Shop,Bakery,Breakfast Spot,Rental Car Location,Residential Building (Apartment / Condo)
4,pe01004,Sushi Restaurant,Coffee Shop,Restaurant,Café,Bakery,Chinese Restaurant,Hostel,Peruvian Restaurant,Plaza,Park


In [83]:
#Normalizar dataframe
#habitantes viviendas vehículos calidad
df['habitantes']=(df['habitantes']-df['habitantes'].min())/(df['habitantes'].max()-df['habitantes'].min())
df['viviendas']=(df['viviendas']-df['viviendas'].min())/(df['viviendas'].max()-df['viviendas'].min())
df['vehículos']=(df['vehículos']-df['vehículos'].min())/(df['vehículos'].max()-df['vehículos'].min())
df['calidad']=(df['calidad']-df['calidad'].min())/(df['calidad'].max()-df['calidad'].min())

df.head()

,id,distribuidor,comuna,latitud,longitud,habitantes,viviendas,vehículos,calidad
0,pe01001,Petrobras,Santiago Centro,-33.453465,-70.644193,0.702481,1.0,0.433635,0.564916
1,pe01002,Petrobras,Santiago Centro,-33.430747,-70.665559,0.702481,1.0,0.433635,0.564916
2,pe01003,Petrobras,Santiago Centro,-33.445162,-70.640700,0.702481,1.0,0.433635,0.564916
3,pe01004,Petrobras,Santiago Centro,-33.442951,-70.636680,0.702481,1.0,0.433635,0.564916
4,pe01006,Petrobras,Santiago Centro,-33.459715,-70.652395,0.702481,1.0,0.433635,0.564916


In [84]:
# set number of clusters
kclusters = 5

distribuidor_grouped = distribuidor_grouped.join(df.set_index('id'), on='ID Station')

distribuidor_grouped_clustering = distribuidor_grouped.drop(labels=['ID Station','distribuidor','comuna','latitud','longitud'], axis=1)

#distribuidor_grouped_clustering

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(distribuidor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=int32)

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

distribuidor_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
distribuidor_merged = distribuidor_merged.join(neighborhoods_venues_sorted.set_index('ID Station'), on='id')

distribuidor_merged.head() # check the last columns!

,id,distribuidor,comuna,latitud,longitud,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,pe01001,Petrobras,Santiago Centro,-33.453465,-70.644193,0.702481,1.0,0.433635,0.564916,0,Grocery Store,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Rock Club,Cupcake Shop,Latin American Restaurant,Fried Chicken Joint,Flea Market
1,pe01002,Petrobras,Santiago Centro,-33.430747,-70.665559,0.702481,1.0,0.433635,0.564916,0,Sushi Restaurant,Soccer Field,Park,Farmers Market,Bakery,River,Chinese Restaurant,BBQ Joint,Skate Park,Antique Shop
2,pe01003,Petrobras,Santiago Centro,-33.445162,-70.640700,0.702481,1.0,0.433635,0.564916,0,Restaurant,Café,Sushi Restaurant,Pizza Place,Plaza,Coffee Shop,Bakery,Breakfast Spot,Rental Car Location,Residential Building (Apartment / Condo)
3,pe01004,Petrobras,Santiago Centro,-33.442951,-70.636680,0.702481,1.0,0.433635,0.564916,0,Sushi Restaurant,Coffee Shop,Restaurant,Café,Bakery,Chinese Restaurant,Hostel,Peruvian Restaurant,Plaza,Park
4,pe01006,Petrobras,Santiago Centro,-33.459715,-70.652395,0.702481,1.0,0.433635,0.564916,0,Restaurant,South American Restaurant,Gym,Music Store,Men's Store,Sporting Goods Shop,Bus Station,Café,General Entertainment,Theater


In [86]:
print(distribuidor_merged.shape)
distribuidor_merged.dropna(axis=0, inplace=True)
print(distribuidor_merged.shape)

(92, 20)
(92, 20)


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(distribuidor_merged['latitud'], 
                                  distribuidor_merged['longitud'], 
                                  distribuidor_merged['id'], 
                                  distribuidor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
distribuidor_merged.loc[distribuidor_merged['Cluster Labels'] == 0, distribuidor_merged.columns[[0] + list(range(5, distribuidor_merged.shape[1]))]]

,id,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,pe01001,0.702481,1.000000,0.433635,0.564916,0,Grocery Store,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Rock Club,Cupcake Shop,Latin American Restaurant,Fried Chicken Joint,Flea Market
1,pe01002,0.702481,1.000000,0.433635,0.564916,0,Sushi Restaurant,Soccer Field,Park,Farmers Market,Bakery,River,Chinese Restaurant,BBQ Joint,Skate Park,Antique Shop
2,pe01003,0.702481,1.000000,0.433635,0.564916,0,Restaurant,Café,Sushi Restaurant,Pizza Place,Plaza,Coffee Shop,Bakery,Breakfast Spot,Rental Car Location,Residential Building (Apartment / Condo)
3,pe01004,0.702481,1.000000,0.433635,0.564916,0,Sushi Restaurant,Coffee Shop,Restaurant,Café,Bakery,Chinese Restaurant,Hostel,Peruvian Restaurant,Plaza,Park
4,pe01006,0.702481,1.000000,0.433635,0.564916,0,Restaurant,South American Restaurant,Gym,Music Store,Men's Store,Sporting Goods Shop,Bus Station,Café,General Entertainment,Theater
5,pe01008,0.702481,1.000000,0.433635,0.564916,0,Sandwich Place,Pizza Place,Chinese Restaurant,Restaurant,Pharmacy,Peruvian Restaurant,Sushi Restaurant,Breakfast Spot,Cosmetics Shop,South American Restaurant
41,pe1311901,0.915480,0.827136,0.562594,0.474274,0,Pharmacy,Farmers Market,Food,Grocery Store,Gas Station,Burger Joint,Deli / Bodega,Sushi Restaurant,Soccer Field,Convenience Store
42,pe1311902,0.915480,0.827136,0.562594,0.474274,0,Convenience Store,Hot Dog Joint,Big Box Store,Pharmacy,Gym,Donut Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
43,pe1311903,0.915480,0.827136,0.562594,0.474274,0,Chinese Restaurant,Breakfast Spot,Food Truck,Nightclub,Sandwich Place,Diner,Restaurant,Supermarket,Mountain,Gastropub
44,pe1311904,0.915480,0.827136,0.562594,0.474274,0,Candy Store,Breakfast Spot,Clothing Store,Auto Garage,Donut Shop,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop


In [61]:
distribuidor_merged.loc[distribuidor_merged['Cluster Labels'] == 1, distribuidor_merged.columns[[0] + list(range(5, distribuidor_merged.shape[1]))]]

,id,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,pe1310201,80832,24547,10800,47.82,1,Hot Dog Joint,Plaza,Big Box Store,Pharmacy,Chinese Restaurant,Yoga Studio,Donut Shop,Food,Flower Shop,Flea Market
7,pe1312101,80832,24547,10800,47.82,1,Ice Cream Shop,Food Truck,Plaza,Restaurant,Farmers Market,Gastropub,Dog Run,Flower Shop,Flea Market,Fish & Chips Shop
8,pe1312602,132622,38020,14100,42.42,1,Construction & Landscaping,Convenience Store,Intersection,Diner,Sushi Restaurant,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market
9,pe1310401,126955,37759,13100,46.52,1,Ice Cream Shop,Department Store,Furniture / Home Store,Donut Shop,Coffee Shop,Pharmacy,Pub,Burger Joint,Bus Station,Sandwich Place
10,pe1310402,126955,37759,13100,46.52,1,Chinese Restaurant,Farmers Market,Pharmacy,Athletics & Sports,Gym,Yoga Studio,Dog Run,Food,Flower Shop,Flea Market
11,pe1310404,126955,37759,13100,46.52,1,Restaurant,Chinese Restaurant,Pharmacy,Gym,Liquor Store,Yoga Studio,Dog Run,Flower Shop,Flea Market,Fish & Chips Shop
12,pe1310501,162505,47941,19500,48.54,1,Ice Cream Shop,Flea Market,Pharmacy,Shopping Mall,Department Store,Bar,Sushi Restaurant,Electronics Store,Food,Flower Shop
13,pe1310502,162505,47941,19500,48.54,1,Bus Station,Chinese Restaurant,Supermarket,Grocery Store,Sandwich Place,Pizza Place,Pharmacy,Intersection,Sushi Restaurant,Donut Shop
14,pe1310601,147041,52486,16600,49.96,1,Park,Liquor Store,Chinese Restaurant,Bar,Bakery,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
15,pe1310602,147041,52486,16600,49.96,1,Bus Station,Bakery,Pharmacy,Hot Dog Joint,Coffee Shop,Performing Arts Venue,Residential Building (Apartment / Condo),Chinese Restaurant,Fast Food Restaurant,Shopping Mall


In [62]:
distribuidor_merged.loc[distribuidor_merged['Cluster Labels'] == 2, distribuidor_merged.columns[[0] + list(range(5, distribuidor_merged.shape[1]))]]

,id,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,pe1311901,521627,161354,77000,55.87,2,Pharmacy,Farmers Market,Food,Grocery Store,Gas Station,Burger Joint,Deli / Bodega,Sushi Restaurant,Soccer Field,Convenience Store
42,pe1311902,521627,161354,77000,55.87,2,Convenience Store,Hot Dog Joint,Big Box Store,Pharmacy,Gym,Donut Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
43,pe1311903,521627,161354,77000,55.87,2,Chinese Restaurant,Breakfast Spot,Food Truck,Nightclub,Sandwich Place,Diner,Restaurant,Supermarket,Mountain,Gastropub
44,pe1311904,521627,161354,77000,55.87,2,Candy Store,Breakfast Spot,Clothing Store,Auto Garage,Donut Shop,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
71,pe1311001,568106,171511,70900,48.60,2,Pharmacy,Soccer Field,Sandwich Place,Fast Food Restaurant,Ice Cream Shop,Gymnastics Gym,Diner,Hardware Store,Movie Theater,Shopping Mall
72,pe1320101,568106,171511,70900,48.60,2,Sandwich Place,Department Store,Chinese Restaurant,Convenience Store,BBQ Joint,Shopping Mall,Soccer Stadium,Pizza Place,Seafood Restaurant,Gastropub
73,sh1320101,568106,171511,70900,48.60,2,Farmers Market,Sushi Restaurant,Liquor Store,Bookstore,Paper / Office Supplies Store,Bus Station,Chinese Restaurant,Bakery,Pharmacy,Fast Food Restaurant


In [63]:
distribuidor_merged.loc[distribuidor_merged['Cluster Labels'] == 3, distribuidor_merged.columns[[0] + list(range(5, distribuidor_merged.shape[1]))]]

,id,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,pe1311401,294838,118007,135000,74.54,3,Arts & Crafts Store,Plaza,Grocery Store,Shopping Mall,Restaurant,Sports Bar,Café,Latin American Restaurant,Sushi Restaurant,Bakery
32,pe1311402,294838,118007,135000,74.54,3,Plaza,Bakery,Pizza Place,Rental Car Location,Chinese Restaurant,Park,Sandwich Place,Bus Station,Farmers Market,Liquor Store
33,pe1311403,294838,118007,135000,74.54,3,Pizza Place,Pharmacy,Sushi Restaurant,Convenience Store,Asian Restaurant,American Restaurant,Soccer Field,Coffee Shop,Peruvian Restaurant,Neighborhood
34,pe1311404,294838,118007,135000,74.54,3,Coffee Shop,Sandwich Place,Hotel,Gift Shop,Restaurant,Gym / Fitness Center,Bridge,Shoe Store,Seafood Restaurant,Spanish Restaurant
35,pe1311405,294838,118007,135000,74.54,3,Sushi Restaurant,Pizza Place,Pub,Dance Studio,Café,Martial Arts Dojo,Chinese Restaurant,Farmers Market,Burger Joint,Building
36,te1311401,294838,118007,135000,74.54,3,Gym / Fitness Center,Athletics & Sports,Gym,Convenience Store,Plaza,Bar,Supermarket,Paper / Office Supplies Store,Restaurant,Tennis Court


In [64]:
distribuidor_merged.loc[distribuidor_merged['Cluster Labels'] == 4, distribuidor_merged.columns[[0] + list(range(5, distribuidor_merged.shape[1]))]]

,id,habitantes,viviendas,vehículos,calidad,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,pe01001,404495,193628,59900,59.27,4,Grocery Store,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Rock Club,Cupcake Shop,Latin American Restaurant,Fried Chicken Joint,Flea Market
1,pe01002,404495,193628,59900,59.27,4,Sushi Restaurant,Soccer Field,Park,Farmers Market,Bakery,River,Chinese Restaurant,BBQ Joint,Skate Park,Antique Shop
2,pe01003,404495,193628,59900,59.27,4,Restaurant,Café,Sushi Restaurant,Pizza Place,Plaza,Coffee Shop,Bakery,Breakfast Spot,Rental Car Location,Residential Building (Apartment / Condo)
3,pe01004,404495,193628,59900,59.27,4,Sushi Restaurant,Coffee Shop,Restaurant,Café,Bakery,Chinese Restaurant,Hostel,Peruvian Restaurant,Plaza,Park
4,pe01006,404495,193628,59900,59.27,4,Restaurant,South American Restaurant,Gym,Music Store,Men's Store,Sporting Goods Shop,Bus Station,Café,General Entertainment,Theater
5,pe01008,404495,193628,59900,59.27,4,Sandwich Place,Pizza Place,Chinese Restaurant,Restaurant,Pharmacy,Peruvian Restaurant,Sushi Restaurant,Breakfast Spot,Cosmetics Shop,South American Restaurant
22,pe1311003,366916,120035,64600,57.23,4,Sushi Restaurant,Pizza Place,Burger Joint,Chinese Restaurant,Fast Food Restaurant,Convenience Store,Pub,Cantonese Restaurant,Restaurant,Supermarket
23,pe1311004,366916,120035,64600,57.23,4,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Burger Joint,Bakery,Restaurant,Park,Sandwich Place,Clothing Store,Pub
24,pe1311005,366916,120035,64600,57.23,4,Pharmacy,Bus Station,Peruvian Restaurant,Chinese Restaurant,Tennis Stadium,Fast Food Restaurant,Metro Station,Soccer Field,Electronics Store,Fried Chicken Joint
25,pe1311006,366916,120035,64600,57.23,4,Performing Arts Venue,Dog Run,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
